In [1]:
import wolframalpha as WA
my_appid = "V4K7V9-KL5PK9LK5V"

In [2]:
client = WA.Client(my_appid)

In [3]:
function_string = "x * y + 50 / x + 20 / y"
#function_string = raw_input()

In [4]:
variables = ['x', 'y', 'z', 't']
variables = list(filter((lambda x : (function_string.find(x) != -1)), variables))

In [5]:
result = client.query("grad " + function_string)

In [6]:
for pod in result.pods:
    if pod.title.find("Result") != -1:
        s = pod.text
        l = s.split('= ')
        s = l[1]
        l = s.split('\n')
        s = l[0]
        if s[0] == '{':
            s = s[1:-1]
        l = s.split(', ')
        derivatives = [l[i] for i in xrange(len(variables))]
        break

In [7]:
derivatives = [s.replace(' ', '*') for s in derivatives]
for oper in ['+', '-', '*', '/', '^']:
    derivatives = [s.replace(oper, " " + oper + " ") for s in derivatives]

In [8]:
#for s, zmienna in zip(derivatives, variables): 
#    print "Pochodna po " + zmienna + " to: " + s

In [9]:
query_cri = "solve "
for i in xrange(len(derivatives) - 1):
    query_cri = query_cri + "(" + derivatives[i] + " = 0" + ") " + "and "
query_cri = query_cri + "(" + derivatives[len(derivatives) - 1] + " = 0" + ")"

res_cri = client.query(query_cri)

In [10]:
critical_points = []
for ple in res_cri.pods:
    if ple.title == "Results" or ple.title == "Result":
        for pleple in list(ple):
            critical_points.append(pleple.text)

In [11]:
#print "Punkty krytyczne:"
#print critical_points

In [12]:
###wyrzucanie zespolonych, powinno działać, chyba że wolfram dostanie downa

critical_points = list(filter((lambda x : x.find('i') == -1), critical_points))

In [13]:
###konwertuj z syntaxy wolframowej do sympy'owej

def convert_power(s):
    return s.replace('^', '**')

def convert_and(s):
    l = s.split('and')
    k = [convert_equal_sign(t) for t in l]
    if len(k) == 1:
        return k[0]
    res = "And("
    for t in k: 
        res = res + t + ","
    res = res[:-1]
    res = res + ")"
    return res

def get_rid_of_approximation(s):
    if s.find("~") == -1:
        return s
    l = s.split("~")
    return l[0]

def convert_equal_sign(s):
    if s.find("=") == -1:
        return s
    s = get_rid_of_approximation(s)
    l = s.split("=")
    return "Eq(" + l[0] + "," + l[1] + ")"
    
function_string = convert_power(function_string)
derivatives = [convert_power(s) for s in derivatives]
critical_points = [convert_power(s) for s in critical_points]
critical_points = [convert_and(s) for s in critical_points]

In [15]:
###latex

from sympy import *
from IPython.display import display
x, y, z, f, e = symbols('x y z f e')
init_printing(use_latex='mathjax')

def equality(A, B):
    return "Eq(" + A + "," + B + ")"

def make_derivative(my_function, my_variable):
    return equality("Derivative(" + fexpr + "," + my_variable + ")", my_function)

fexpr = "f("
for var in variables:
    fexpr = fexpr + var + ","
fexpr = fexpr[:-1] + ")"

print "Podsumowanie:"
print ""
print "Moja funkcja to "
display(eval(equality(fexpr, function_string)))
print ""
for s, var in zip(derivatives, variables): 
    print "Pochodna po " + var + " to: "
    display(eval(make_derivative(s, var)))
print ""

if len(critical_points) == 0 :
    print "Brak punktów krytycznych ᕦ( ͡° ͜ʖ ͡°)ᕤ"
else :
    print "Punkty krytyczne to:"
    for s in critical_points:
        display(eval(s))

Podsumowanie:

Moja funkcja to 


                20   50
f(x, y) = x⋅y + ── + ──
                y    x 


Pochodna po x to: 


∂                 50
──(f(x, y)) = y - ──
∂x                 2
                  x 

Pochodna po y to: 


∂                 20
──(f(x, y)) = x - ──
∂y                 2
                  y 


Punkty krytyczne to:


x = 5 ∧ y = 2